In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc/wrappers/TinyMPC-julia/codegen_examples/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
import ForwardDiff as FD
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

import MeshCat as mc
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

include(joinpath(@__DIR__,"utils/cartpole_animation.jl"))

using Plots
# using Printf

In [3]:
# Define ground truth cartpole ODE for simulation
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    return [qd;qdd]

end

function rk4(params::NamedTuple, x::Vector,u,dt::Float64)
    k1 = dt*dynamics(params, x, u)
    k2 = dt*dynamics(params, x + k1/2, u)
    k3 = dt*dynamics(params, x + k2/2, u)
    k4 = dt*dynamics(params, x + k3, u)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end

rk4 (generic function with 1 method)

In [6]:
# Define problem parameters
params = (mc = 1.2, mp = 0.16, l = 0.55)

# states and control sizes 
nx = 4 
nu = 1 

# desired x and u
xgoal = [0, pi, 0, 0]
ugoal = [0]

# initial condition
x0 = xgoal + [10, 0, 0, 0]

# simulation size 
dt = 0.01
tf = 10.0 
t_vec = 0:dt:tf
Nsim = length(t_vec)+100
Nh = 10 # horizon length

# define cost matrices
Q = diagm([1,1,0.05,1])
Qf = 10*Q
R = [0.1]
R = reshape(R,(nu,nu))
ρ = 0.1

# Determine linearized dynamics of the cartpole about its upright position
A = FD.jacobian(dx -> rk4(params, dx, ugoal, dt), xgoal)
B = FD.jacobian(du -> rk4(params, xgoal, du, dt), ugoal)

xmin = -5 .* ones((nx,Nh))
xmax = 5 .* ones((nx,Nh))
umin = -5 .* ones((nu,Nh))
umax = 5 .* ones((nu,Nh))

# Solver options
abs_pri_tol = 1e-3
rel_pri_tol = 1e-3
max_iter = 100
verbose = 1

tinympc_dir = "/home/sam/Git/tinympc/TinyMPC/"
output_dir = "/generated_code"

"/generated_code"

In [37]:
# Create wrappers for the C backend
using Libdl
# create pointer to shared library
lib = dlopen(tinympc_dir * output_dir * "/build/tinympc/libtinympc.so")

# create pointers to tiny_wrapper functions
set_x = dlsym(lib, "set_x")
function ccall_set_x!(x, verbose)
    @ccall $set_x(x::Ptr{Cfloat}, verbose::Int64)::Cvoid 
end

set_x0 = dlsym(lib, "set_x0")
function ccall_set_x!(x0, verbose)
    @ccall $set_x0(x0::Ptr{Cfloat}, verbose::Int64)::Cvoid 
end

set_xref = dlsym(lib, "set_xref")
function ccall_set_xref!(xref, verbose)
    @ccall $set_xref(xref::Ptr{Cfloat}, verbose::Int64)::Cvoid 
end

reset_dual_variables = dlsym(lib, "reset_dual_variables")
function ccall_reset_dual_variables!(verbose)
    @ccall $reset_dual_variables(verbose::Int64)::Cvoid
end

call_tiny_solve = dlsym(lib, "call_tiny_solve")
function ccall_tiny_solve!()
    @ccall $call_tiny_solve()::Cint
end

get_x = dlsym(lib, "get_x")
function ccall_get_x!(x_soln, verbose)
    @ccall $get_x(x_soln::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

get_u = dlsym(lib, "get_u")
function ccall_get_u!(u_soln, verbose)
    @ccall $get_u(u_soln::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_xmin = dlsym(lib, "set_xmin")
function ccall_set_xmin!(xmin, verbose)
    @ccall $set_xmin(xmin::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_xmax = dlsym(lib, "set_xmax")
function ccall_set_xmax!(xmax, verbose)
    @ccall $set_xmax(xmax::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_umin = dlsym(lib, "set_umin")
function ccall_set_umin!(umin, verbose)
    @ccall $set_umin(umin::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

set_umax = dlsym(lib, "set_umax")
function ccall_set_umax!(umax, verbose)
    @ccall $set_umax(umax::Ptr{Cfloat}, verbose::Int64)::Cvoid
end

ErrorException: could not load symbol "set_x":
/home/sam/Git/tinympc/TinyMPC//generated_code/build/tinympc/libtinympc.so: undefined symbol: set_x

In [32]:
# # tests
# @show x0
# ccall_set_x!(x0)
# ccall_reset_dual_variables!()
# ccall_tiny_solve!()

xgoal = Array{Float32}(xgoal)
ccall_set_xref!(xgoal, 0)

xmin = [-100,-100,-100,-100]
xmax = [100,100,100,100]
xmin = Array{Float32}(xmin)
xmax = Array{Float32}(xmax)

umin = [-100]
umax = [100]
umin = Array{Float32}(umin)
umax = Array{Float32}(umax)


# x_soln = Array{Float32}(x_soln)
# ccall_print_x!(x_soln)
# @show Array{Float32}(x_soln)

1-element Vector{Float32}:
 100.0

In [40]:
# MPC loop

x_hist = [Array{Float32}(zeros(nx)) for i = 1:Nsim] # store x values for sim
Nmax = 1
for k=1:Nsim
    # set initial condition and enforce float type
    x0 = vec(x0)
    x0 = Array{Float32}(x0)
    x_hist[k] = 1*x0
    ccall_set_x!(x0, 0)
    x0 = Array{Float32}(x0)

    ccall_set_xref!(xgoal, 0)

    ccall_set_xmin!(xmin, 0)
    ccall_set_xmax!(xmax, 0)

    ccall_set_umin!(umin, 0)
    ccall_set_umax!(umax, 0)

    # reset dual variables
    ccall_reset_dual_variables!(0)

    # solve
    exitflag = ccall_tiny_solve!()

    # get u solution
    u_soln = zeros((nu,Nh-1))
    u_soln = Array{Float32}(u_soln)
    ccall_get_u!(u_soln, 1)
    u_soln = Array{Float32}(u_soln)
    uk = Array{Float32}(u_soln)[1,1]
    
    xk = rk4(params,x0,uk,dt)
    x0 = 1*xk
    @show Array{Float32}(xk)
end

u_soln:  nan
u_soln:  nan
u_soln:  nan
u_soln:  nan
u_soln:  nan
u_soln:  nan
u_soln:  nan
u_soln:  nan
u_soln:  nan
tiny solve finished


DomainError: DomainError with Inf:
sin(x) is only defined for finite x.

In [36]:
# Simulate
display(animate_cartpole(x_hist[1:Nsim], dt))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


DomainError: DomainError with Inf:
sin(x) is only defined for finite x.